<a href="https://colab.research.google.com/github/hyunjin2123/BA_ToDo/blob/main/gs25%2C%EB%82%A0%EC%94%A8_0607.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%matplotlib inline
import matplotlib as mpl               # 기본 설정
import seaborn as sns           # Seaborn 로딩하기
import matplotlib.pyplot as plt        # 그래프 관련
import matplotlib.font_manager as fm   # 폰트 관련
import pandas as pd
import numpy as np
import lightgbm                         #LGBM 활용 - 기존 tree 기반 알고리즘과 달리 tree구조가 수직적으로 확장 *GBM이란 틀린부분에 가중치를 더하면서 진행하는 알고리즘
from tqdm import tqdm
import warnings

mpl.rcParams['axes.unicode_minus'] = False  ## 마이너스 부호가 깨지는 것 방지

#한글 사용하는 방법 !! 
#1. 먼저 나눔 폰트 설치
#2. 전체 폰트 갯수 중  나눔 폰트 갯수가 뜨는지 확인
#3. 런타임-런타임 다시 시작 후, 차트 plot 다시 실행

!apt-get update -qq
!apt-get install fonts-nanum* -qq              

sys_font=fm.findSystemFonts()
print(f"sys_font number: {len(sys_font)}")
print(sys_font)                               

nanum_font = [f for f in sys_font if 'Nanum' in f]
print(f"nanum_font number: {len(nanum_font)}") 
nanum_font

In [49]:
# 설치된 나눔글꼴중 원하는 폰트의 전체 경로 가져오기 
path = '/usr/share/fonts/truetype/nanum/NanumSquareB.ttf'  
font_name = fm.FontProperties(fname=path, size=14).get_name()
print(font_name)

plt.rc('font', family=font_name)

NanumSquare


In [50]:
# 간단하게 그림을 그릴 수 있는 mglearn 라이브러리 사용 (!pip install mglearn 명령어로 설치)
!pip install mglearn
import mglearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# 경고 끄기
pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings(action='ignore')

In [51]:
#GS25 상품 판매 데이터 불러오기

from pandas.core.frame import DataFrame

import pandas as pd

# csv 데이터 가져오기
df_GS25 = pd.read_csv("/content/drive/My Drive/Colab Notebooks/BigMining/최종과제/GS25.csv", encoding="cp949")

print(df_GS25.columns)
print(df_GS25.info())      #데이터정보파악 행, 열, 결측치 등
display(df_GS25.head(5))    #처음 5행 확인

Index(['korea_cvs.pvn_nm', 'korea_cvs.sale_dt', 'korea_cvs.gen_cd',
       'korea_cvs.age_cd', 'korea_cvs.category', 'korea_cvs.adj_qty',
       'korea_cvs.bor_nm'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2707786 entries, 0 to 2707785
Data columns (total 7 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   korea_cvs.pvn_nm    object
 1   korea_cvs.sale_dt   int64 
 2   korea_cvs.gen_cd    object
 3   korea_cvs.age_cd    object
 4   korea_cvs.category  object
 5   korea_cvs.adj_qty   int64 
 6   korea_cvs.bor_nm    object
dtypes: int64(2), object(5)
memory usage: 144.6+ MB
None


,korea_cvs.pvn_nm,korea_cvs.sale_dt,korea_cvs.gen_cd,korea_cvs.age_cd,korea_cvs.category,korea_cvs.adj_qty,korea_cvs.bor_nm
0,서울특별시,20160101,F,00~19,라면,7,종로구
1,서울특별시,20160101,F,20~39,과자,655,종로구
2,서울특별시,20160101,F,20~39,라면,282,종로구
3,서울특별시,20160101,F,20~39,마스크,35,종로구
4,서울특별시,20160101,F,20~39,맥주,183,종로구


In [52]:
df_GS25.columns=['광역시','ymd','성별','연령대','상품','수량','시군구']
df_GS25

,광역시,ymd,성별,연령대,상품,수량,시군구
0,서울특별시,20160101,F,00~19,라면,7,종로구
1,서울특별시,20160101,F,20~39,과자,655,종로구
2,서울특별시,20160101,F,20~39,라면,282,종로구
3,서울특별시,20160101,F,20~39,마스크,35,종로구
4,서울특별시,20160101,F,20~39,맥주,183,종로구
...,...,...,...,...,...,...,...
2707781,경기도,20181230,M,40~59,과자,7,여주시이천시
2707782,경기도,20181231,F,00~19,맥주,7,여주시이천시
2707783,경기도,20181231,F,20~39,생리대,7,여주시이천시
2707784,경기도,20181231,F,60~99,생리대,7,여주시이천시


In [53]:
#종로구 자료만 선택
df_GS25_temp = df_GS25[df_GS25['시군구'] == '종로구'] # 시군구 열이 '종로구'인 행들만 추출
df_GS25_temp

,광역시,ymd,성별,연령대,상품,수량,시군구
0,서울특별시,20160101,F,00~19,라면,7,종로구
1,서울특별시,20160101,F,20~39,과자,655,종로구
2,서울특별시,20160101,F,20~39,라면,282,종로구
3,서울특별시,20160101,F,20~39,마스크,35,종로구
4,서울특별시,20160101,F,20~39,맥주,183,종로구
...,...,...,...,...,...,...,...
2519452,서울특별시,20181231,M,00~19,면도기,7,종로구
2519453,서울특별시,20181231,M,20~39,생리대,7,종로구
2519454,서울특별시,20181231,M,20~39,스타킹,7,종로구
2519455,서울특별시,20181231,M,40~59,마스크,7,종로구


In [54]:
df_GS25_temp = df_GS25_temp.drop(['광역시','성별','연령대','시군구'], axis=1)
df_GS25_temp

,ymd,상품,수량
0,20160101,라면,7
1,20160101,과자,655
2,20160101,라면,282
3,20160101,마스크,35
4,20160101,맥주,183
...,...,...,...
2519452,20181231,면도기,7
2519453,20181231,생리대,7
2519454,20181231,스타킹,7
2519455,20181231,마스크,7


In [58]:
df_GS25_temp.groupby(['ymd','상품']).수량.sum()

ymd       상품   
20160101  과자       1500
          라면        818
          마스크        56
          맥주        499
          면도기         7
                   ... 
20181231  생수        353
          숙취해소제       7
          스타킹        35
          아이스크림     239
          탄산음료      126
Name: 수량, Length: 11978, dtype: int64

In [59]:
product = pd.DataFrame(df_GS25_temp.groupby(['ymd','상품']).수량.sum()).reset_index()
df_GS25_final = product.pivot('ymd','상품','수량').fillna(0)
df_GS25_final

상품,과자,라면,마스크,맥주,면도기,생리대,생수,숙취해소제,스타킹,아이스크림,우산,탄산음료
ymd,,,,,,,,,,,,
20160101,1500.0,818.0,56.0,499.0,7.0,85.0,274.0,35.0,28.0,514.0,0.0,98.0
20160102,1536.0,718.0,21.0,514.0,0.0,141.0,281.0,49.0,21.0,366.0,0.0,126.0
20160103,1437.0,853.0,14.0,380.0,42.0,77.0,345.0,21.0,28.0,578.0,0.0,105.0
20160104,1297.0,973.0,7.0,282.0,14.0,113.0,217.0,14.0,14.0,401.0,0.0,147.0
20160105,1395.0,937.0,0.0,360.0,7.0,77.0,253.0,105.0,35.0,408.0,0.0,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...
20181227,1008.0,1289.0,21.0,289.0,14.0,42.0,239.0,49.0,14.0,415.0,0.0,98.0
20181228,1536.0,1212.0,35.0,380.0,14.0,77.0,359.0,126.0,21.0,296.0,0.0,133.0
20181229,1289.0,746.0,21.0,268.0,7.0,49.0,401.0,42.0,21.0,232.0,0.0,49.0


In [60]:
print(df_GS25_final.describe())

상품              과자           라면          마스크           맥주          면도기  \
count  1096.000000  1096.000000  1096.000000  1096.000000  1096.000000   
mean   1482.708029  1079.035584    26.665146   383.551095    12.824818   
std     291.678929   199.204655    47.159460   124.088586    11.071225   
min     662.000000   592.000000     0.000000    98.000000     0.000000   
25%    1268.000000   944.000000     7.000000   295.000000     7.000000   
50%    1473.000000  1070.500000    14.000000   366.000000    14.000000   
75%    1662.000000  1211.000000    28.000000   451.000000    21.000000   
max    2620.000000  2558.000000   621.000000  1021.000000    70.000000   

상품             생리대           생수        숙취해소제          스타킹        아이스크림  \
count  1096.000000  1096.000000  1096.000000  1096.000000  1096.000000   
mean     86.135949   522.141423    85.363139    42.876825   967.811131   
std      31.355973   211.904401    48.608693    34.611719   656.202305   
min       7.000000   154.000000     0

In [22]:
#기상청 기상 데이터 불러오기

# csv 데이터 가져오기
df_wether = pd.read_csv("/content/drive/My Drive/Colab Notebooks/BigMining/최종과제/기상데이터.csv", encoding="cp949")

print(df_wether.columns)
print(df_wether.info())      #데이터정보파악 행, 열, 결측치 등
display(df_wether.head(5))      #처음 5행 확인

Index(['bigcon_weather.tm', 'bigcon_weather.stn_id', 'bigcon_weather.pvn_nm',
       'bigcon_weather.bor_nm', 'bigcon_weather.max_ta',
       'bigcon_weather.max_ws', 'bigcon_weather.min_ta',
       'bigcon_weather.avg_ta', 'bigcon_weather.avg_rhm',
       'bigcon_weather.avg_ws', 'bigcon_weather.sum_rn'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59113 entries, 0 to 59112
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   bigcon_weather.tm       59113 non-null  int64  
 1   bigcon_weather.stn_id   59113 non-null  int64  
 2   bigcon_weather.pvn_nm   59113 non-null  object 
 3   bigcon_weather.bor_nm   59113 non-null  object 
 4   bigcon_weather.max_ta   58826 non-null  float64
 5   bigcon_weather.max_ws   58778 non-null  float64
 6   bigcon_weather.min_ta   58826 non-null  float64
 7   bigcon_weather.avg_ta   58814 non-null  float64
 8   bigcon_weather.avg_rhm  43

,bigcon_weather.tm,bigcon_weather.stn_id,bigcon_weather.pvn_nm,bigcon_weather.bor_nm,bigcon_weather.max_ta,bigcon_weather.max_ws,bigcon_weather.min_ta,bigcon_weather.avg_ta,bigcon_weather.avg_rhm,bigcon_weather.avg_ws,bigcon_weather.sum_rn
0,20160101,98,경기도,동두천시,6.5,3.8,-6.4,-0.2,74.0,0.9,0.0
1,20160102,98,경기도,동두천시,10.1,3.2,1.9,5.4,70.0,1.3,0.0
2,20160103,98,경기도,동두천시,9.0,2.4,2.1,4.7,82.0,0.7,0.0
3,20160104,98,경기도,동두천시,8.0,5.3,-3.0,1.9,55.0,2.2,0.0
4,20160105,98,경기도,동두천시,3.1,5.6,-6.4,-2.8,36.0,1.8,0.0


In [23]:
df_wether_temp = df_wether.drop(columns = ['bigcon_weather.stn_id','bigcon_weather.max_ta','bigcon_weather.max_ws','bigcon_weather.min_ta'])
df_wether_temp.head()

,bigcon_weather.tm,bigcon_weather.pvn_nm,bigcon_weather.bor_nm,bigcon_weather.avg_ta,bigcon_weather.avg_rhm,bigcon_weather.avg_ws,bigcon_weather.sum_rn
0,20160101,경기도,동두천시,-0.2,74.0,0.9,0.0
1,20160102,경기도,동두천시,5.4,70.0,1.3,0.0
2,20160103,경기도,동두천시,4.7,82.0,0.7,0.0
3,20160104,경기도,동두천시,1.9,55.0,2.2,0.0
4,20160105,경기도,동두천시,-2.8,36.0,1.8,0.0


In [24]:
df_wether_temp.columns=['ymd','광역시','시군구','평균기온','습도','평균풍속','강수량']
df_wether_temp

,ymd,광역시,시군구,평균기온,습도,평균풍속,강수량
0,20160101,경기도,동두천시,-0.2,74.0,0.9,0.0
1,20160102,경기도,동두천시,5.4,70.0,1.3,0.0
2,20160103,경기도,동두천시,4.7,82.0,0.7,0.0
3,20160104,경기도,동두천시,1.9,55.0,2.2,0.0
4,20160105,경기도,동두천시,-2.8,36.0,1.8,0.0
...,...,...,...,...,...,...,...
59108,20181227,경기도,연천군,-10.2,35.0,1.6,0.0
59109,20181228,경기도,연천군,-12.2,48.0,1.1,0.0
59110,20181229,경기도,연천군,-9.3,46.0,1.2,0.0
59111,20181230,경기도,연천군,-10.3,56.0,0.8,0.0


In [25]:
#기상데이터도 종로구 자료만 선택
df_wether_final = df_wether_temp[df_wether_temp['시군구'] == '종로구'] # 시군구 열이 '종로구'인 행들만 추출
df_wether_final.head(10)

,ymd,광역시,시군구,평균기온,습도,평균풍속,강수량
2192,20160101,서울특별시,종로구,1.2,73.0,1.6,0.0
2193,20160102,서울특별시,종로구,5.7,77.0,2.0,0.0
2194,20160103,서울특별시,종로구,6.5,81.0,1.8,0.0
2195,20160104,서울특별시,종로구,2.0,54.0,3.1,0.0
2196,20160105,서울특별시,종로구,-2.7,39.0,2.3,0.0
2197,20160106,서울특별시,종로구,-1.7,54.0,1.8,0.0
2198,20160107,서울특별시,종로구,-3.4,52.0,2.5,0.0
2199,20160108,서울특별시,종로구,-3.3,50.0,2.0,0.0
2200,20160109,서울특별시,종로구,-2.1,57.0,2.1,0.0
2201,20160110,서울특별시,종로구,0.3,42.0,2.6,0.0


In [26]:
df_wether_final = df_wether_final.drop(['광역시','시군구',], axis=1)
df_wether_final.tail()

,ymd,평균기온,습도,평균풍속,강수량
3283,20181227,-10.1,28.0,3.5,0.0
3284,20181228,-10.5,33.0,2.5,0.0
3285,20181229,-7.6,29.0,2.9,0.0
3286,20181230,-6.8,36.0,1.2,0.0
3287,20181231,-5.4,40.0,1.2,0.0


In [27]:
df_wether_final = df_wether_final.fillna(0)
df_wether_final.isnull().sum()

ymd     0
평균기온    0
습도      0
평균풍속    0
강수량     0
dtype: int64

In [78]:
#gs 데이터와 기상 데이터 결합하기

df = pd.merge( df_wether_final, df_GS25_final, on="ymd", how="left")
df.info()  #1096개 데이터로 3년 모두 포함되어 있음을 확인
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1096 entries, 0 to 1095
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ymd     1096 non-null   int64  
 1   평균기온    1096 non-null   float64
 2   습도      1096 non-null   float64
 3   평균풍속    1096 non-null   float64
 4   강수량     1096 non-null   float64
 5   과자      1096 non-null   float64
 6   라면      1096 non-null   float64
 7   마스크     1096 non-null   float64
 8   맥주      1096 non-null   float64
 9   면도기     1096 non-null   float64
 10  생리대     1096 non-null   float64
 11  생수      1096 non-null   float64
 12  숙취해소제   1096 non-null   float64
 13  스타킹     1096 non-null   float64
 14  아이스크림   1096 non-null   float64
 15  우산      1096 non-null   float64
 16  탄산음료    1096 non-null   float64
dtypes: float64(16), int64(1)
memory usage: 154.1 KB


,ymd,평균기온,습도,평균풍속,강수량,과자,라면,마스크,맥주,면도기,생리대,생수,숙취해소제,스타킹,아이스크림,우산,탄산음료
0,20160101,1.2,73.0,1.6,0.0,1500.0,818.0,56.0,499.0,7.0,85.0,274.0,35.0,28.0,514.0,0.0,98.0
1,20160102,5.7,77.0,2.0,0.0,1536.0,718.0,21.0,514.0,0.0,141.0,281.0,49.0,21.0,366.0,0.0,126.0
2,20160103,6.5,81.0,1.8,0.0,1437.0,853.0,14.0,380.0,42.0,77.0,345.0,21.0,28.0,578.0,0.0,105.0
3,20160104,2.0,54.0,3.1,0.0,1297.0,973.0,7.0,282.0,14.0,113.0,217.0,14.0,14.0,401.0,0.0,147.0
4,20160105,-2.7,39.0,2.3,0.0,1395.0,937.0,0.0,360.0,7.0,77.0,253.0,105.0,35.0,408.0,0.0,77.0


##데이터 탐색


In [30]:
# 기술통계 확인하기
print(df.describe())

                ymd         평균기온          습도         평균풍속          강수량  \
count  1.096000e+03  1096.000000  1096.00000  1096.000000  1096.000000   
mean   2.017066e+07    13.223358    58.05292     2.067792     3.167427   
std    8.178333e+03    11.166435    14.53030     0.708135    12.686887   
min    2.016010e+07   -14.800000    22.00000     0.000000     0.000000   
25%    2.016098e+07     3.475000    47.00000     1.600000     0.000000   
50%    2.017070e+07    14.650000    58.00000     2.000000     0.000000   
75%    2.018040e+07    22.900000    68.00000     2.500000     0.000000   
max    2.018123e+07    33.700000    97.00000     5.400000   144.500000   

                과자           라면          마스크           맥주          면도기  \
count  1096.000000  1096.000000  1096.000000  1096.000000  1096.000000   
mean   1482.708029  1079.035584    26.665146   383.551095    12.824818   
std     291.678929   199.204655    47.159460   124.088586    11.071225   
min     662.000000   592.000000     0

In [66]:
# 상관계수 확인하기 1 - 매트릭스 형태
corr_df = df.corr()
corr_df = corr_df.apply(lambda x: round(x ,2))
corr_df

,ymd,평균기온,습도,평균풍속,강수량,과자,라면,마스크,맥주,면도기,생리대,생수,숙취해소제,스타킹,아이스크림,우산,탄산음료
ymd,1.00,-0.01,-0.04,-0.32,0.03,-0.47,-0.22,0.10,-0.18,-0.32,-0.18,-0.09,-0.18,-0.26,-0.11,-0.06,-0.03
평균기온,-0.01,1.00,0.41,-0.19,0.15,0.05,-0.14,-0.18,0.48,0.05,0.18,0.46,-0.04,-0.11,0.82,0.12,0.40
습도,-0.04,0.41,1.00,-0.13,0.45,0.09,-0.03,-0.07,0.18,0.03,0.03,0.05,-0.02,-0.16,0.19,0.40,0.03
평균풍속,-0.32,-0.19,-0.13,1.00,0.04,0.20,0.13,0.01,-0.05,0.05,0.03,-0.11,0.01,0.15,-0.09,0.01,-0.08
강수량,0.03,0.15,0.45,0.04,1.00,0.04,0.01,-0.09,0.08,-0.05,-0.07,-0.12,-0.07,-0.10,-0.00,0.38,-0.09
과자,-0.47,0.05,0.09,0.20,0.04,1.00,0.23,-0.00,0.35,0.16,0.10,0.20,0.16,0.09,0.22,0.12,0.12
라면,-0.22,-0.14,-0.03,0.13,0.01,0.23,1.00,0.05,-0.04,0.08,0.09,0.09,0.24,0.32,-0.13,0.09,0.24
마스크,0.10,-0.18,-0.07,0.01,-0.09,-0.00,0.05,1.00,-0.17,-0.03,-0.02,-0.02,0.09,0.15,-0.19,-0.06,0.03
맥주,-0.18,0.48,0.18,-0.05,0.08,0.35,-0.04,-0.17,1.00,0.07,0.16,0.35,0.05,-0.12,0.52,0.06,0.24
면도기,-0.32,0.05,0.03,0.05,-0.05,0.16,0.08,-0.03,0.07,1.00,0.10,0.07,0.07,0.08,0.11,0.04,0.02


In [67]:
# 상관계수 확인하기 2 - 시리즈 형태
s = corr_df.unstack()
s

ymd   ymd      1.00
      평균기온    -0.01
      습도      -0.04
      평균풍속    -0.32
      강수량      0.03
               ... 
탄산음료  숙취해소제    0.17
      스타킹      0.18
      아이스크림    0.37
      우산      -0.01
      탄산음료     1.00
Length: 289, dtype: float64

In [68]:
# 상관계수 확인하기 3 - DataFrame의 style프로퍼티 형태
df = pd.DataFrame(s[s < 1].sort_values(ascending=False), columns=['corr'])
df.style.background_gradient(cmap='viridis')         # 0.4 이상을 유의미하다고 봄

,,corr
아이스크림,평균기온,0.820000
평균기온,아이스크림,0.820000
아이스크림,맥주,0.520000
맥주,아이스크림,0.520000
평균기온,맥주,0.480000
맥주,평균기온,0.480000
생수,평균기온,0.460000
평균기온,생수,0.460000
습도,강수량,0.450000
강수량,습도,0.450000


##Scitiy-learn

판매수량 합계 컬럼 하나 추가해서 총판매량 예측해보기!

In [61]:
# Python ≥3.5 이상을 권장
import sys
print("Python version: {}".format(sys.version))

# Scikit-Learn ≥0.20 이상을 권장
import sklearn
print("scikit-learn version: {}".format(sklearn.__version__))

Python version: 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
scikit-learn version: 1.0.2


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

%matplotlib inline
import matplotlib.pyplot as plt

# 데이터 위에서 만든 데이터셋 사용

# 데이터 준비
df = pd.DataFrame(df, columns= df.feature_names)
df['총판매량'] = df.target #예측하고자 하는 가격 변수를 price라는 이름으로 추가 
display(df.head())

X= df[['습도', '강수량']] # 범죄 발생률(%),모집단의 하위계층의 비율(%)
y= df['총판매량']           # 주택 가격

# 데이터 시각화
plt.scatter(X['습도'], y)
plt.xlabel("습도")
plt.ylabel("총판매량")
plt.show()

plt.scatter(X['강수량'], y)
plt.xlabel("강수량")
plt.ylabel("총판매량")
plt.show()

# 선형 모델 선택
model = LinearRegression()

# 모델 훈련  
model.fit(X,y) 
print('절편 값: ', model.intercept_)
print('회귀 계수 값: ', np.round(model.coef_, 1))   


# 주택 가격 예측
X_new = [[50, 50]]            # 습도 50, 강수량 50 일 때 판매량 예측
print(model.predict(X_new))       

## 의사결정나무 (Decision tree) 모델

In [99]:
df = df.astype('float')     # y_train 데이터 타입 오류

In [100]:
train_df = df.drop(['습도','평균풍속','강수량','과자','마스크'], axis=1)
test_df = df.drop(['습도','평균풍속','강수량','과자','마스크'], axis=1)

X_train = train_df.drop('평균기온', axis=1)
y_train = train_df['평균기온']
X_test = test_df

In [101]:
print(X_train.shape)
print(y_train.shape)   # y_train 데이터 타입 오류
print(X_test.shape)

(1096, 11)
(1096,)
(1096, 12)


In [98]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [93]:
params = {
    'max_depth': [2, 3, 5, 10],
    'min_samples_split': [2, 3, 5],
    'min_samples_leaf': [1, 5, 8]
}

In [94]:
dt_clf = DecisionTreeClassifier()

In [90]:
train_df = train_df.astype({'평균기온': 'int'})

In [103]:
grid_dclf = GridSearchCV(dt_clf, param_grid=params, scoring='accuracy', cv=5)
grid_dclf.fit(X_train, y_train)

ValueError: ignored

In [104]:
from sklearn.ensemble import GradientBoostingClassifier
# 기본값은 max_depth= 3, n_estimators=100, learning_rate = 0.1

gbrt = GradientBoostingClassifier(random_state=0)
gbrt.fit(X_train, y_train)

ValueError: ignored